This file uses the halo Critical_R_200 to validate the algorithm designed by us.

In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import FunctionLib
#%matplotlib widget
basePath ='./sims.TNG/TNG50-1/output'
from tqdm import tqdm
import gc
import os


In [2]:
m_dm=4.5E-5  #E10M_Sum
h=0.6774  #km/Mpc
Critial_Density=1.27E-8  #E10M_Sum/kpc^3
Critial_Density_h=Critial_Density/h**2  #E10M_Sum/(kpc/h)^3


In [3]:
gas_fields=['Coordinates','Masses','Velocities']
dm_fields=['Coordinates','Velocities']
stars_fields=['Coordinates','Masses','Velocities']
bh_fields=['Coordinates','Masses','Velocities']

halo_fields=['GroupPos','Group_M_Crit200','Group_R_Crit200','GroupMass']


In [10]:
def Cal_M_200_From_R_200(R_200,Comoving_Critial_Density):

    M_200=200*Comoving_Critial_Density*4/3*3.14*R_200**3
    return M_200

def R_200_Calculation(snapshot_num,halos_data,halo_id,Comoving_Critial_Density):

    subhalo_comoving_position=halos_data['GroupPos'][halo_id]
    subhalo_mass=halos_data['GroupMass'][halo_id]

    gas_data_tensor=il.snapshot.loadHalo(basePath,snapshot_num,halo_id,0,fields=gas_fields)
    dm_data_tensor=il.snapshot.loadHalo(basePath,snapshot_num,halo_id,1,fields=dm_fields)
    stars_data_tensor=il.snapshot.loadHalo(basePath,snapshot_num,halo_id,4,fields=stars_fields)
    bh_data_tensor=il.snapshot.loadHalo(basePath,snapshot_num,halo_id,5,fields=bh_fields)


    precision = 0.1
    max_iter = 100
    left, right = 0, 1000

    for i in range(max_iter):

        mid = (left + right) / 2

        mass_in=total_mass_in_radius(gas_data_tensor,dm_data_tensor,stars_data_tensor,bh_data_tensor,subhalo_comoving_position,mid)

        density=3*mass_in/(4*3.14*mid**3)

        if density > 200*Comoving_Critial_Density:
            left = mid

        else:
            right = mid


        if abs(right - left) < precision or i == max_iter - 1:
            R_200 = (left + right) / 2
            break

    M_200=Cal_M_200_From_R_200(R_200,comoving_critial_density)

    return R_200,M_200,mass_in

def baryon_mass_in_radius(Center,Radius,Coordinates,Masses):
    dist_to_com = np.sqrt(np.sum((Coordinates - Center)**2, axis=1))
    within_radius_idx = np.where(dist_to_com <= Radius)[0]
    total_mass_within_radius = np.sum(Masses[within_radius_idx])
    return total_mass_within_radius

def dm_mass_in_radius(Center,Radius,Coordinates):
    dist_to_com = np.sqrt(np.sum((Coordinates - Center)**2, axis=1))
    within_radius_idx = np.where(dist_to_com <= Radius)[0]
    total_mass_within_radius = m_dm*within_radius_idx.shape[0]*h
    return total_mass_within_radius

def total_mass_in_radius(gas_data_tensor,dm_data_tensor,stars_data_tensor,bh_data_tensor,Center,Radius):

    gas_mass_in=baryon_mass_in_radius(Center,Radius,gas_data_tensor['Coordinates'],gas_data_tensor['Masses'])
    dm_mass_in=dm_mass_in_radius(Center,Radius,dm_data_tensor['Coordinates'])
    stars_mass_in=baryon_mass_in_radius(Center,Radius,stars_data_tensor['Coordinates'],stars_data_tensor['Masses'])
    bh_mass_in=baryon_mass_in_radius(Center,Radius,bh_data_tensor['Coordinates'],bh_data_tensor['Masses'])

    total_mass_in=gas_mass_in+dm_mass_in+stars_mass_in+bh_mass_in

    return total_mass_in


In [11]:
for i in range(70,80):
    halos_data=il.groupcat.loadHalos(basePath,i,fields=halo_fields)
    header=il.groupcat.loadHeader(basePath, i)

    scale_factor=header['Time']
    comoving_critial_density=Critial_Density_h*(scale_factor**2)
    for j in range(20,30):
        R_200,M_200,m_200=R_200_Calculation(i,halos_data,j,comoving_critial_density)
        print(R_200,halos_data['Group_R_Crit200'][j],end=' ')
        print(M_200,m_200,halos_data['Group_M_Crit200'][j])
    print(' ')


406.768798828125 398.529 751.882525222 751.933846602 794.893
352.813720703125 345.481 490.618015004 490.625578726 517.763
412.139892578125 402.354 782.061763711 782.014265156 817.978
343.719482421875 335.208 453.648563293 453.649729613 473.001
377.044677734375 366.791 598.805163593 598.717650506 619.697
343.536376953125 335.473 452.923949522 452.960500887 474.095
328.826904296875 320.847 397.199894035 397.238601041 414.768
328.338623046875 320.023 395.43309143 395.467880032 411.561
340.484619140625 332.715 440.960396806 441.013367912 462.505
341.705322265625 335.188 445.720205511 445.718109828 472.912
 


KeyboardInterrupt: 

In [12]:
def Velocity_Virial_Calculation(R_200,M_200,Critial_Density):
    Unit=4.3E4
    V=np.sqrt(G*R_200/M_200*Unit)

